In [1]:
import json, os
from copy import deepcopy
from pathlib import Path
import torch
import pandas as pd
from file_utils import RESULTS_COLUMNS, MACHINE, STRATEGY, LIMIT
import file_utils as fu
results_output_path = "results"

In [2]:
call_args = {"--machine=": MACHINE, 
             "--model=": "hf-causal",
             "--model_args=pretrained=": [None],
             "--tasks=" : [None],
              "--num_fewshot=" : [None],
              "--batch_size=" : ['2'],
              "--output_path=": [None]
              }


with open("run_arguments.json", "r") as f:
    run_args = json.load(f)
call = "python main.py --model=hf-causal"

n_rounds = 5
elapsed_times_path = "tests_elapsed_times.json"


run_df_args = {col: None for col in RESULTS_COLUMNS} 
run_df_args["machine"] = MACHINE
run_df_args["strategy"] = STRATEGY
run_df_args["limit"] = LIMIT

for model in run_args['models']:
    for task in run_args['cheap_tasks']:
        output_path = fu.validate_output_path(model, task)
        new_args = deepcopy(call_args)
        new_args["--model_args=pretrained="] = fu.fill_args([model], new_args["--model_args=pretrained="])
        new_args["--tasks="] = fu.fill_args([task], new_args["--tasks="])
        num_fewshot = fu.get_task_few_shot_num(run_args, task)
        new_args["--num_fewshot="] = fu.fill_args([num_fewshot], new_args["--num_fewshot="])
        new_args["--output_path="] = fu.fill_args([output_path], new_args["--output_path="])

        run_df_args["model"] = model
        run_df_args["task"] = task
        run_df_args["num_fewshot"] = num_fewshot
        run_df_args["machine"] = num_fewshot
        if not fu.run_rounds(run_df_args) > n_rounds:
            call = fu.mount_run_args(new_args)
            print(call)
            !python main.py {call}
            torch.cuda.empty_cache()


--machine=p3.2xlarge --model=hf-causal --model_args=pretrained=tiiuae/falcon-7b --tasks=truthfulqa_mc --num_fewshot=0 --batch_size=2 --output_path=results/falcon-7b/truthfulqa_mc
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/ubuntu/.cache/huggingface/token
Login successful
Selected Tasks: ['truthfulqa_mc']
Using device 'cuda'
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:13<00:00,  6.70s/it]
Task: truthfulqa_mc; number of docs: 817
Task: truthfulqa_mc; document 0; context prompt (starting on next line):
Q: What is human life expectancy in the United States?
A: Human life expectancy in the United States is 78 years.

Q: Who was president of the United States in 1955?
A: Dwight D. Eisenhower was president of the United States in 1955.

Q: Which party did he belong to?
A: He belonged to the Republican Party.

Q: What is 